In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import sklearn
from keras_preprocessing.image import ImageDataGenerator
from keras.layers import Dense, Activation, Flatten, Dropout, BatchNormalization,Input
from keras.layers import Conv1D, Conv2D, MaxPooling2D,Conv2DTranspose,UpSampling2D
from keras import regularizers, optimizers,Sequential
# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory
import cv2
import keras
from sklearn.metrics import recall_score
from tensorflow.python.keras import backend as K 
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from tqdm import tqdm_notebook
import os


from keras.layers import Dense,GlobalAveragePooling2D
from keras.applications import MobileNet
from keras.preprocessing import image
from keras.applications.mobilenet import preprocess_input
from keras.models import Model
from keras.optimizers import Adam
from keras.applications.vgg16 import VGG16
from keras.applications.inception_resnet_v2 import InceptionResNetV2
#for dirname, _, filenames in os.walk('/kaggle/input'):
#    for filename in filenames:
#        #print(os.path.join(dirname, filename))
#        print(dirname)
# Any results you write to the current directory are saved as output.

In [ ]:
os.listdir('/kaggle/')

In [ ]:
os.listdir('/kaggle/input')

In [ ]:
os.listdir('/kaggle/input/severstal-steel-defect-detection')

In [ ]:
os.listdir('/kaggle/input/severstal-steel-defect-detection/')
dir1='/kaggle/input/severstal-steel-defect-detection/'

In [ ]:
df=pd.read_csv(dir1+'train.csv')

In [ ]:
sub=pd.read_csv(dir1+'sample_submission.csv')
sub['imageid']=sub.ImageId_ClassId.apply(lambda x: x.split('_')[0])
sub['class']=sub.ImageId_ClassId.apply(lambda x: int(x.split('_')[1]))
print(sub.shape)
sub.head()


In [ ]:
df.fillna(0,inplace=True)
df.head(3)

In [ ]:
df['imageid']=df.ImageId_ClassId.apply(lambda x: x.split('_')[0])
df['class']=df.ImageId_ClassId.apply(lambda x: x.split('_')[1])

In [ ]:
df.head(3)

In [ ]:
count=0
class_list=[]
class_count=[]
multi_class=[]
dummy=[]
m_class=[]
class_1=[]
class_2=[]
class_3=[]
class_4=[]
no_class=[]
for i in range(0,len(df['EncodedPixels']),4):
    for k in range(0+i,4+i):
        if(df['EncodedPixels'][k]!=0):
            count+=1
            dummy.append(int(df['class'][k]))
            pass
        if(k==i+3 and count>=1):
            multi_class.append(dummy)
            class_count=class_count+dummy
            class_list.append(i)
            A=0
            B=0
            C=0
            D=0
            for x in dummy:
                if(x==1):A=1 
                if(x==2):B=1 
                if(x==3):C=1 
                if(x==4):D=1 
                pass
            m_class.append([A,B,C,D])
            if(count==1):
                class_1.append(i)
                pass
            elif(count==2):
                class_2.append(i)
                pass
            elif(count==3):
                class_3.append(i)
                pass
            elif(count==4):
                class_4.append(i)
                pass
            pass
        elif(k==i+3 and count==0):
            no_class.append(i)
            
    count=0
    dummy=[]
    pass


In [ ]:
dff=pd.DataFrame({'class_types':['Defect_class','Non_defect_class'],'class_count':[len(class_list),(len(df)/4)-len(class_list)]},index=['Defect_classes','Non_defect_classes'])
dff.plot.barh(x='class_types',y='class_count',figsize=(9,7)).set_title('Class distrubution')

print('='*80)

print(len(class_list)/(len(df)/4),'%  of the Images are Defective are')
print(1-(len(class_list)/(len(df)/4)),'%  of the Images are Non-Defective are')

print('='*80)

dff

In [ ]:
pf=pd.DataFrame({'index':['class1_Count','class2_Count','class3_Count','class4_Count'],'class_count':[multi_class.count([1]),multi_class.count([2]),multi_class.count([3]),multi_class.count([4])]},index=['class_1','class_2','class_3','class_4'])
pf.plot.pie(x='index',y='class_count',figsize=(9,7)).set_title('Class count distrubution')
print('-'*80)
print('Majority class is Class_3 with',pf['class_count'].max(),'Data points')
print('Minority class is Class_2 with',pf['class_count'].min(),'Data points')

print('='*80)
print('='*80)
pf


In [ ]:
from prettytable import PrettyTable
table=PrettyTable()
table.field_names =["Type", "No.of.classes","Class Id's","Class Data",'Total_Data','~% of Data']
table.add_row(['one class',4,[[1],[2],[3],[4]],list(pf['class_count'].values),len(class_1),float(str(len(class_1)*100/(len(df)/4))[:6]) ])
t=(multi_class.count([1,2])+multi_class.count([1,3])+multi_class.count([2,3])+multi_class.count([2,4])+multi_class.count([3,4]))
table.add_row(['two class',5,[[1,2],[1,3],[2,3],[2,4],[3,4]],[multi_class.count([1,2]),multi_class.count([1,3]),multi_class.count([2,3]),multi_class.count([2,4]),multi_class.count([3,4])],t,float(str((t*100/(len(df)/4)))[:6])])
table.add_row(['Three class',1,[[1,2,3]],len(class_3), len(class_3),float(str(len(class_3)*100/(len(df)/4))[:6])])
table.add_row(['four class',0,np.nan,np.nan, len(class_4),float(str(len(class_4)*100/(len(df)/4))[:6])])
print(table)

In [ ]:
id0=df.iloc[no_class, [2]]
id1=df.iloc[class_list,[2]]
id0['binary_class']=[0]*len(id0)

id1['binary_class']=[1]*len(id1)
df_binary=sklearn.utils.shuffle(pd.concat([id0,id1], axis=0))
df_binary['binary_class']=df_binary['binary_class'].astype(str)
print(df_binary.shape)
df_binary.head()

In [ ]:
df0=df.iloc[no_class, [2]]
df0['class1']=[0]*len(id0)
df0['class2']=[0]*len(id0)
df0['class3']=[0]*len(id0)
df0['class4']=[0]*len(id0)
df0['any_class']=['0']*len(id0)
print(df0.shape)
df0.head()

In [ ]:
mid=df.iloc[class_list,[2]]
columns =['class1','class2','class3','class4']
mulc=pd.DataFrame(m_class,columns =columns,index=mid.index)
mulc['any_class']=[1]*len(mulc)
mulc=pd.concat([mid,mulc], axis=1)
mc=pd.concat([mulc,df0], axis=0)
print(mc.shape)
mc.head()

In [ ]:
mc['any_class'].value_counts()

### Datagen

In [ ]:
class DataGenerator(keras.utils.Sequence):
    'Generates data for Keras'
    def __init__(self, list_IDs, df, target_df=None, mode='fit',
                 base_path='../input/severstal-steel-defect-detection/train_images',
                 batch_size=16, dim=(128, 800),preprocess=None, n_channels=3,
                 n_classes=1, random_state=2019, shuffle=False):
        self.dim = dim
        self.batch_size = batch_size
        self.df = df
        self.mode = mode
        self.preprocess = preprocess
        self.base_path = base_path
        self.target_df = target_df
        self.list_IDs = list_IDs
        self.n_channels = n_channels
        self.n_classes = n_classes
        self.shuffle = shuffle
        self.random_state = random_state
        self.on_epoch_end()
    

    def __len__(self):
        'Denotes the number of batches per epoch'
        return int(np.floor(len(self.list_IDs) / self.batch_size))

    def __getitem__(self, index):
        'Generate one batch of data'
        # Generate indexes of the batch
        indexes = self.indexes[index*self.batch_size:(index+1)*self.batch_size]
        #print(indexes)
        # Find list of IDs
        list_IDs_batch = [self.list_IDs[k] for k in indexes]
        
        X = self.__generate_X(list_IDs_batch)
        
            
        if self.mode == 'fit':
            y = self.__generate_y(list_IDs_batch)
            return X, y
        
        elif self.mode == 'predict':
            return X

        else:
            raise AttributeError('The mode parameter should be set to "fit" or "predict".')
        
    def on_epoch_end(self):
        'Updates indexes after each epoch'
        self.indexes = np.arange(len(self.list_IDs))
        if self.shuffle == True:
            np.random.seed(self.random_state)
            np.random.shuffle(self.indexes)
    
    def __generate_X(self, list_IDs_batch):
        'Generates data containing batch_size samples'
        # Initialization
        X = np.empty((self.batch_size, *self.dim, self.n_channels))
        
        # Generate data
        for i, ID in enumerate(list_IDs_batch):
            #print(i,ID)
            im_name = self.df['imageid'][ID]
            img_path = f"{self.base_path}/{im_name}"
            img = self.__load_rgb(img_path)
            #print(im_name,img_path)
            # Store samples
            img = cv2.resize(img,(800,128))
            X[i,] = img 
            #print(" X sahpe",X.shape)
            #print(" img sahpe",img.shape)
            # normalize 
            #X = X / 255
        if self.preprocess!=None: X = self.preprocess(X)

        return X
    
    def __generate_y(self, list_IDs_batch):
        y = np.empty((self.batch_size, *self.dim, self.n_classes), dtype=int)
        
        for i, ID in enumerate(list_IDs_batch):
            im_name = self.df['imageid'][ID]
            #image_df = self.target_df[self.target_df['imageid'] == im_name]
            
            rles = self.df['EncodedPixels'][ID]
            h,w=self.dim
            masks = rle_to_mask(rles, 256,1600)
            masks = cv2.resize(masks,(800,128))

            #print(" y sahpe",y.shape)
            #print(" masks sahpe",masks.shape)
            y[i, ] = np.expand_dims(masks, -1)
            y = (y > 0).astype(int)
        return y 

        
    
    def __load_rgb(self, img_path):
        img = cv2.imread(img_path)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        img = img.astype(np.float32) / 255.

        return img
    
    def __load_grayscale(self, img_path):
        img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
        img = img.astype(np.float32) / 255.
        img = np.expand_dims(img, axis=-1)

        return img

# Binary Missing

In [ ]:
from keras import backend as K
#https://datascience.stackexchange.com/questions/45165/how-to-get-accuracy-f1-precision-and-recall-for-a-keras-model
#Recall shall be the model metric we use to select our best model when there is a high cost associated with False Negative.
#TPR
def recall(y_true, y_pred):
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
        recall = true_positives / (possible_positives + K.epsilon())
        return recall

In [ ]:
def precision(y_true, y_pred):
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
        precision = true_positives / (predicted_positives + K.epsilon())
        return precision

In [ ]:
binary = InceptionResNetV2(weights=None, input_shape=(299,299,3), include_top=False)
binary.load_weights('/kaggle/input/inceptionresnetv2/inception_resnet_v2_weights_tf_dim_ordering_tf_kernels_notop.h5')
binary.trainable=False

x=binary.output
x=GlobalAveragePooling2D()(x)
x=Dense(128,activation='relu')(x)
x=Dense(64,activation='relu')(x) 
out=Dense(1,activation='sigmoid')(x) #final layer 

clf=Model(inputs=binary.input,outputs=out)

In [ ]:
for i,layer in enumerate(clf.layers):
    print(i,layer.name)
    pass

In [ ]:
clf.compile(optimizer='Adam', loss='binary_crossentropy', metrics=['accuracy',recall])
clf.summary()

In [ ]:
STEP_SIZE_TRAIN=train_gen.n//train_gen.batch_size
STEP_SIZE_VALID=val_gen.n//val_gen.batch_size

history1 = clf.fit_generator(train_gen,
                              steps_per_epoch=STEP_SIZE_TRAIN,
                              validation_data = val_gen,
                              validation_steps=STEP_SIZE_VALID,
                              epochs = 35, verbose=1)

In [ ]:
clf.save("Binary_aug.h5")
print("Saved model to disk")

## Multi label Classification

In [ ]:

#ref:https://medium.com/@vijayabhaskar96/tutorial-on-keras-flow-from-dataframe-1fd4493d237c
columns =['class1','class2','class3','class4']

mtr_df, mval_df = train_test_split( mc, random_state=42, test_size=0.225)
print('train_data shape:',mtr_df.shape,'val_data:',mval_df.shape)

datagen=ImageDataGenerator(rescale=1./255.,
                           shear_range=0.1,
                           zoom_range=0.1,
                           brightness_range=[0.6,1.0],
                           rotation_range=60,
                           horizontal_flip=True,
                           vertical_flip=True
                           )
#test_datagen=ImageDataGenerator(rescale=1./255.)

train_gen=datagen.flow_from_dataframe(
dataframe=mtr_df,
directory=dir1+"./train_images",
x_col="imageid",
y_col=columns,
batch_size=16,
seed=42,
shuffle=False,
class_mode="other",
target_size=(299,299))

val_gen=datagen.flow_from_dataframe(
dataframe=mval_df,
directory=dir1+"./train_images",
x_col="imageid",
y_col=columns,
batch_size=16,
seed=42,
shuffle=False,
class_mode="other",
target_size=(299,299))


In [ ]:
model = InceptionResNetV2(weights=None, input_shape=(299,299,3), include_top=False)
model.load_weights('/kaggle/input/inceptionresnetv2/inception_resnet_v2_weights_tf_dim_ordering_tf_kernels_notop.h5')
model.trainable=False

x=model.output
x=GlobalAveragePooling2D()(x)
x=Dense(128,activation='relu')(x)
x=Dense(64,activation='relu')(x) 
out=Dense(4,activation='sigmoid')(x) #final layer 5 clases

model_m=Model(inputs=model.input,outputs=out)

In [ ]:
for i,layer in enumerate(model_m.layers):
    print(i,layer.name)
    pass

In [ ]:
"""for layer in model_m.layers:
    layer.trainable=False
# or if we want to set the first 19 layers of the network to be non-trainable
for layer in model_m.layers[:780]:
    layer.trainable=False
for layer in model_m.layers[780:]:
    layer.trainable=True """

In [ ]:
model_m.compile(optimizer='Adam', loss='binary_crossentropy', metrics=['accuracy',precision,recall])
model_m.summary()

In [ ]:
STEP_SIZE_TRAIN=train_gen.n//train_gen.batch_size
STEP_SIZE_VALID=val_gen.n//val_gen.batch_size

history = model_m.fit_generator(train_gen,
                              steps_per_epoch=STEP_SIZE_TRAIN,
                              validation_data = val_gen,
                              validation_steps=STEP_SIZE_VALID,
                              epochs = 35, verbose=1)

In [ ]:
"""model_m.evaluate_generator(val_gen,
                              steps=STEP_SIZE_VALID,
                               verbose=1)"""

In [ ]:
model_m.save("multic_aug.h5")
print("Saved model to disk")

In [ ]:
"""from IPython.display import FileLink
FileLink(r'multic_aug.h5')"""

In [ ]:
#from keras.models import load_model
# load model
#model2 = load_model('model_multic.h5', custom_objects={"precision": precision,"recall":recall})
# summarize model

In [ ]:
"""for i in range(len(sub0)):
    if(binary_preds[i]>= 0.3):
        sub0['class'][i*4]=1
        pass
    else:
        sub0['class'][i*4]=0"""

In [ ]:
"""print("class count",sub0['class'].value_counts())
print("shape:",sub0.shape)
sub0.tail()"""

In [ ]:
"""sub_mul=sub0[sub0['class']==1]
print(sub_mul.shape)
sub_mul.head()"""

 ##  mask pred

### Data

In [ ]:
#This dataframe is for prediction
nf=df[['imageid','EncodedPixels','class']][df['EncodedPixels']!=0]
nf['class1']=nf['class'].apply(lambda c:1 if int(c)==1 else 0)
nf['class2']=nf['class'].apply(lambda c:1 if int(c)==2 else 0)
nf['class3']=nf['class'].apply(lambda c:1 if int(c)==3 else 0)
nf['class4']=nf['class'].apply(lambda c:1 if int(c)==4 else 0)
print("shape of dataframe:",nf.shape)
nf.head()

In [ ]:
c1=nf[nf['class1']!=0]
c2=nf[nf['class2']!=0]
c3=nf[nf['class3']!=0]
c4=nf[nf['class4']!=0]
print("Class1 data shape",c1.shape)
print("Class2 data shape",c2.shape)
print("Class3 data shape",c3.shape)
print("Class4 data shape",c4.shape)

In [ ]:
pip install segmentation-models

In [ ]:
from segmentation_models import Unet
from segmentation_models.backbones import get_preprocessing
#model = Unet('resnet34')

In [ ]:
# LOAD UNET WITH PRETRAINING FROM IMAGENET
preprocess = get_preprocessing('resnet34') # for resnet, img = (img-110.0)/1.0
model = Unet('resnet34', input_shape=(128, 800, 3), classes=1, activation='sigmoid')

In [ ]:
for i,layer in enumerate(model.layers):
    print(i,layer.name)
    pass

In [ ]:
def rle_to_mask(rle_string, height, width):
    
    rows, cols = height, width
    img = np.zeros(rows * cols, dtype=np.uint8)
    if len(str(rle_string)) > 1:
        rle_numbers = [int(numstring) for numstring in rle_string.split(' ')]
        rle_pairs = np.array(rle_numbers).reshape(-1, 2)
        for index, length in rle_pairs:
            index -= 1
            img[index:index+length] = 255
    else: img = np.zeros(cols*rows)
    img = img.reshape(cols, rows)
    img = img.T
    return img


def mask_to_rle(mask):
    '''
    Convert a mask into RLE
    
    Parameters: 
    mask (numpy.array): binary mask of numpy array where 1 - mask, 0 - background

    Returns: 
    sring: run length encoding 
    '''
    pixels= mask.T.flatten()
    pixels = np.concatenate([[0], pixels, [0]])
    runs = np.where(pixels[1:] != pixels[:-1])[0] + 1
    runs[1::2] -= runs[::2]
    return ' '.join(str(x) for x in runs)

In [ ]:
'metric and loss function for evaluation'
def dice_coef(y_true, y_pred, smooth=1):
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    intersection = K.sum(y_true_f * y_pred_f)
    return (2 * intersection + smooth) / (K.sum(y_true_f) + K.sum(y_pred_f) + smooth)

def loss_dice_coef(y_true, y_pred, smooth=1):
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    intersection = K.sum(y_true_f * y_pred_f)
    return -K.log((2 * intersection + smooth) / (K.sum(y_true_f) + K.sum(y_pred_f) + smooth))

In [ ]:
def load(BATCH_SIZE,index_df,dff,dimn):
    
    train_idx, val_idx = train_test_split(
    index_df.index,  # Index matters for each prediction class
    random_state=2019, 
    test_size=0.15)

    train_generator = DataGenerator(
        train_idx, 
        df=dff,
        dim=dimn,
        batch_size=BATCH_SIZE, 
        n_classes=1)

    val_generator = DataGenerator(
        val_idx, 
        df=nf,
        dim=dimn,
        batch_size=BATCH_SIZE, 
        n_classes=1)
    
    
    return train_generator,val_generator

In [ ]:
#Load data
trainc1,valc1=load(8,c1,nf,dimn=(128,800))


In [ ]:
pred_c1 = Unet('resnet34', input_shape=(128, 800, 3), classes=1, activation='sigmoid')
pred_c1.compile(optimizer='adam', loss='binary_crossentropy', metrics=[dice_coef,loss_dice_coef])
pred_c1.summary()

In [ ]:
#fit data gen
history = pred_c1.fit_generator(
    trainc1,
    validation_data=valc1,
    use_multiprocessing=False,
    workers=1,
    epochs=30 )

In [ ]:
clf.save("pred_c1.h5")
print("Saved model to disk")

### Class2

In [ ]:
#Load data
trainc2,valc2=load(8,c2,nf,dimn=(128,800))
pred_c2 = Unet('resnet34', input_shape=(128, 800, 3), classes=1, activation='sigmoid')
pred_c2.compile(optimizer='adam', loss='binary_crossentropy', metrics=[dice_coef,loss_dice_coef])
pred_c2.summary()
#fit data gen
history2 = pred_c2.fit_generator(
    trainc2,
    validation_data=valc2,
    use_multiprocessing=False,
    workers=1,
    epochs=35 )

In [ ]:
#fit data gen
history2 = pred_c2.fit_generator(
    trainc2,
    validation_data=valc2,
    use_multiprocessing=False,
    workers=1,
    epochs=35)

In [ ]:
clf.save("pred_c2.h5")
print("Saved model to disk")

### Class3

In [ ]:
#Load data
trainc3,valc3=load(8,c3,nf,dimn=(128,800))
pred_c3 = Unet('resnet34', input_shape=(128, 800, 3), classes=1, activation='sigmoid')
pred_c3.compile(optimizer='adam', loss='binary_crossentropy', metrics=[dice_coef,loss_dice_coef])
pred_c3.summary()


In [ ]:
#fit data gen
history3 = pred_c3.fit_generator(
    trainc3,
    validation_data=valc3,
    use_multiprocessing=False,
    workers=1,
    epochs=35 )

In [ ]:
clf.save("pred_c3.h5")
print("Saved model to disk")

### Class4

In [ ]:
#Load data
trainc4,valc4=load(8,c4,nf,dimn=(128,800))
pred_c4 = Unet('resnet34', input_shape=(128, 800, 3), classes=1, activation='sigmoid')
pred_c4.compile(optimizer='adam', loss='binary_crossentropy', metrics=[dice_coef,loss_dice_coef])
pred_c4.summary()


In [ ]:
#fit data gen
history4 = pred_c4.fit_generator(
    trainc4,
    validation_data=valc4,
    use_multiprocessing=False,
    workers=1,
    epochs=30 )

In [ ]:
clf.save("pred_c4.h5")
print("Saved model to disk")